In [ ]:
import numpy as np # 행렬
import scipy as sp # 과학기술계산
import pandas as pd # 행렬기반 데이터처리
import matplotlib.pyplot as plt # 간단한 plotting
import seaborn as sns # 다양한 plotting #Jupyter notebook 쓸 때 %matplotlib inline씀
%matplotlib inline
import statsmodels.api as sm #모델패키지
import sklearn as sk #모델패키지
import math
import colorama
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, mean_absolute_error, mean_absolute_percentage_error
# plt.rc('font', family='NanumBarunGothic') # 한글폰트설정

In [ ]:
#Data set 설정
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

def Data_func():
    lawdata=pd.read_csv("240902_rawdata_doe_fea_results_1.csv", encoding='cp949') #xp949는 'utf-8' codec can't~ 에러 발생할 때
    X = lawdata.iloc[:,0:4].values
    y = lawdata.iloc[:,5:10].values
    
    Xp_train,Xp_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=77)
    scaler = preprocessing.RobustScaler()  #<<<<<< Scaler
    X_train = scaler.fit_transform(Xp_train) # train data는 fit_transform()을 쓰고
    X_test = scaler.transform(Xp_test) # test data는 transform()을 씀 → train으로 fit한 데이터 기준으로 test 데이터 정규화
    return (X_train, y_train), (X_test, y_test), (Xp_train)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
#ANN 모델 설정; kernel, activation func, optimizer 설정
def ANNmodel(num_of_hidden_layers,num_of_hidden_layer_nodes,learning_rate):
    model = Sequential()
    for i in range(0,num_of_hidden_layers):
        if i == 0:
            model.add(Dense(input_dim=4,units=num_of_hidden_layer_nodes, kernel_initializer='glorot_normal',activation='tanh'))
        else:
            model.add(Dense(units=num_of_hidden_layer_nodes, kernel_initializer='glorot_normal', activation='tanh'))
            #activation; sigmoid, tanh, ReLU, LeakyReLU, ELU
    #Adding output layer
    model.add(Dense(units=5,kernel_initializer='glorot_normal'))
    
    opt = keras.optimizers.Adagrad(learning_rate) #<<<<<<<<< Optimizer (SGD, Adagrad, Adadelta, Adamax, RMSprop, Adam, Nadam)
    model.compile(optimizer=opt,loss='mean_squared_error')#,metrics=['r2_score'])
    
    model.summary()
    return model

In [ ]:
#최적화된 Hyperparameter 값 정리
num_of_hidden_layers = 2 #hyperparam opt 에서 Nh 값
num_of_hidden_layer_nodes = round(87.80423258505886) #hyperparam opt 에서 Nhn 값
bat_size = round(1.20776077166901) #hyperparam opt 에서 batch_size 값
learning_rate = 0.08573696384661184 #hyperparam opt 에서 learning_rate 값

num_epochs = 500 #참고논문에서 500 고정

In [ ]:
# 데이터 로딩 및 모델 생성
(X_train, y_train), (X_test, y_test), (Xp_train) = Data_func()
model = ANNmodel(num_of_hidden_layers, num_of_hidden_layer_nodes, learning_rate)

# 모델 학습
history = model.fit(
    X_train, y_train,
    epochs=num_epochs,
    batch_size=bat_size,
    verbose=0,
    validation_data=(X_test, y_test)
)

# 손실 값 가져오기
loss_total = history.history['loss']
valid = history.history['val_loss']

# 손실 그래프 그리기
plt.plot(loss_total)
# plt.plot(valid)  # 검증 손실을 보고 싶다면 이 줄을 주석 해제
# plt.ylim([1, 100])  # 필요한 경우 범위 설정
plt.yscale('log')
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Opt.case'], loc='upper right')
# plt.legend(['Train', 'Test'], loc='upper left')  # 두 그래프 모두 그릴 경우
plt.show()

# 학습 마지막 손실값 출력
loss_f = loss_total[-1]
print(loss_f)

# 손실 결과 저장
df = pd.DataFrame(loss_total)
df.to_excel(r"C:/Users/admin/0_ANN/240808_tierod/240904_trial_1/240909_loss_total_Adagrad_tanh_opt_rawdata_t10.xlsx", index=False)

In [ ]:
#모델 검증, train, test 인풋값 대비 예측값 비교
y_predict_train = model.predict(X_train)
y_predict_test = model.predict(X_test)

plt.plot(y_train[:,0], y_predict_train[:,0], 'or', label='train') #색깔 b,g,r,c,m,y,k,w
plt.plot(y_test[:,0], y_predict_test[:,0], 'ob', label='test')

plt.xlim([175,350]) #x축 범위 설정
plt.ylim([175,350]) #y축 범위설정

plt.title('Fitting')
plt.xlabel('FEA caulking load(kN)')
plt.ylabel('Predict caulking load(kN)')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

mape_disp_train = mean_absolute_percentage_error(y_train[:,0], y_predict_train[:,0])
r2_disp_train = r2_score(y_train[:,0], y_predict_train[:,0])
mape_disp_test = mean_absolute_percentage_error(y_test[:,0], y_predict_test[:,0])
r2_disp_test = r2_score(y_test[:,0], y_predict_test[:,0])

print('MAPE train =',mape_disp_train)
print('MAPE test =',mape_disp_test)
print('R2 train =',r2_disp_train)
print('R2 test =',r2_disp_test)

#----------------------------------------------------------------------

plt.plot(y_train[:,1], y_predict_train[:,1], 'oc', label='train')
plt.plot(y_test[:,1], y_predict_test[:,1], 'om', label='test')

plt.xlim([20,120])
plt.ylim([20,120])

plt.title('Fitting')
plt.xlabel('FEA contact pressure(MPa)')
plt.ylabel('Predict contact pressure(MPa)')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

mape_weight_train = mean_absolute_percentage_error(y_train[:,1], y_predict_train[:,1])
r2_weight_train = r2_score(y_train[:,1], y_predict_train[:,1])
mape_weight_test = mean_absolute_percentage_error(y_test[:,1], y_predict_test[:,1])
r2_weight_test = r2_score(y_test[:,1], y_predict_test[:,1])

print('MAPE train =',mape_weight_train)
print('MAPE test =',mape_weight_test)
print('R2 train =',r2_weight_train)
print('R2 test =',r2_weight_test)

#----------------------------------------------------------------------
plt.plot(y_train[:,2], y_predict_train[:,2], 'oc', label='train')
plt.plot(y_test[:,2], y_predict_test[:,2], 'om', label='test')

plt.xlim([0,3.5])
plt.ylim([0,3.5])

plt.title('Fitting')
plt.xlabel('FEA tilting torque(Nm)')
plt.ylabel('Predict tilting torque(Nm)')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

mape_weight_train = mean_absolute_percentage_error(y_train[:,2], y_predict_train[:,2])
r2_weight_train = r2_score(y_train[:,2], y_predict_train[:,2])
mape_weight_test = mean_absolute_percentage_error(y_test[:,2], y_predict_test[:,2])
r2_weight_test = r2_score(y_test[:,2], y_predict_test[:,2])

print('MAPE train =',mape_weight_train)
print('MAPE test =',mape_weight_test)
print('R2 train =',r2_weight_train)
print('R2 test =',r2_weight_test)
#----------------------------------------------------------------------
plt.plot(y_train[:,3], y_predict_train[:,3], 'oc', label='train')
plt.plot(y_test[:,3], y_predict_test[:,3], 'om', label='test')

plt.xlim([0,4.5])
plt.ylim([0,4.5])

plt.title('Fitting')
plt.xlabel('FEA rotational torque(Nm)')
plt.ylabel('Predict rotational torque(Nm)')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

mape_weight_train = mean_absolute_percentage_error(y_train[:,3], y_predict_train[:,3])
r2_weight_train = r2_score(y_train[:,3], y_predict_train[:,3])
mape_weight_test = mean_absolute_percentage_error(y_test[:,3], y_predict_test[:,3])
r2_weight_test = r2_score(y_test[:,3], y_predict_test[:,3])

print('MAPE train =',mape_weight_train)
print('MAPE test =',mape_weight_test)
print('R2 train =',r2_weight_train)
print('R2 test =',r2_weight_test)
#----------------------------------------------------------------------
plt.plot(y_train[:,4], y_predict_train[:,4], 'oc', label='train')
plt.plot(y_test[:,4], y_predict_test[:,4], 'om', label='test')

plt.xlim([50,90])
plt.ylim([50,90])

plt.title('Fitting')
plt.xlabel('FEA pull-out load(kN)')
plt.ylabel('Predict pull-out load(kN)')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

mape_weight_train = mean_absolute_percentage_error(y_train[:,4], y_predict_train[:,4])
r2_weight_train = r2_score(y_train[:,4], y_predict_train[:,4])
mape_weight_test = mean_absolute_percentage_error(y_test[:,4], y_predict_test[:,4])
r2_weight_test = r2_score(y_test[:,4], y_predict_test[:,4])

print('MAPE train =',mape_weight_train)
print('MAPE test =',mape_weight_test)
print('R2 train =',r2_weight_train)
print('R2 test =',r2_weight_test)
#----------------------------------------------------------------------
#rawdata 저장
results_train = np.concatenate((y_train,y_predict_train),axis=1)
results_test = np.concatenate((y_test,y_predict_test),axis=1)

df = pd.DataFrame(results_train, columns = ['tr_CL','tr_CP','tr_TT','tr_RT','tr_PL','p_CL','p_CP','p_TT','p_RT','p_PL'])
#tr:train,te:test,p:predict,W:weight,CL:caulking load,CP:contact pressure,TT:tilting torque,RT:rotational torque,PL:pullout load
df.to_excel(r"C:/Users/admin/0_ANN/240808_tierod/240904_trial_1/240904_y_train_predict_rawdata_t10.xlsx", index=False)
df = pd.DataFrame(results_test, columns = ['te_CL','te_CP','te_TT','te_RT','te_PL','p_CL','p_CP','p_TT','p_RT','p_PL'])
df.to_excel(r"C:/Users/admin/0_ANN/240808_tierod/240904_trial_1/240904_y_test_predict_rawdata_t10.xlsx", index=False)

In [ ]:
#모델 성능 평가
performance_test = model.evaluate(X_test, y_test, batch_size=1)

print(performance_test)

In [ ]:
#모델 저장
from keras.models import load_model
model.save('240904_tierod_model_t10.h5')